# Análise Shark Attack
## Objetivo: Incremento de Segurança das Praias

## Perguntas a serem respondidas:

#### 1) Quais horários os ataques acontecem mais? Há algum horário em que devemos aumentar a vigilância e/ou reforçar a equipe de guarda-costas das praias?
#### 2) É relevante uma atenção adicional por gênero (M / F) de banhistas?
#### 3) E por país?


## Premissas

#### Iremos trabalhar com os dados de tempo (coluna Time), Gênero (coluna Sex) e País (coluna Country), limpando os dados e porteriormente analisando a base. 

In [1]:
#Primeiro importamos as lybraries:
import pandas as pd
import numpy as np
import re

In [2]:
#Em seguida, importamos o dataframe a ser utilizado: "attacks.csv"
df = pd.read_csv('attacks.csv', encoding='latin-1')
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Buscamos as primeiras informações gerais sobre o dataframe.
# A base é bem incompleta. A coluna que mais tem registros tem 8702 registros, bem abaixo do total de 25722 registros totais.
# Depois de definir a pergunta sobre o case, percebemos que a qtde de registros é pequena então só teremos definições sobre 12%
# da base aproximadamente.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [4]:
# Contamos os valores usando Value Counts e vemos os 3 principais horários em que os ataques ocorrem: 11h, 12h e 15h.
# Notamos também a grande quantidade de registros apenas como "Afternoon"(Tarde) e "Morning" (Manhã).
# Percebemos ainda a necessidade de limpar bem a coluna antes de trabalhar pois existemm muitos valores despadronizados
# (com a mistura de horários tanto numéricos quanto em texto. Ex. "Early evening").

df['Time'].value_counts(normalize=True)


Afternoon    0.063433
11h00        0.043419
Morning      0.041045
12h00        0.036974
15h00        0.036635
               ...   
             0.000339
16h23        0.000339
11h06        0.000339
15h01        0.000339
15h56        0.000339
Name: Time, Length: 366, dtype: float64

In [5]:
df['Time'].unique()

# Usando Unique vemos os valores sem repetição e analisando-os, decidimos usar o Replace para classifios valores em 4 grupos: 
# Morning, Late Morning, Afternoon e Late Afternoon. Incluímos mais alguns grupos que ainda sobraram com qtdes significantes. 
# Assim, ao invés de um horário específico, conseguimos entender qual período tem maior incidência de acidentes.  

array(['18h00', '14h00  -15h00', '07h45', nan, 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '15h30', '07h30', '19h00, Dusk', 'Night', '16h00',
       '15h01', '12h00', '13h45', '23h30', '09h00', '14h30', '18h30',
       '12h30', '16h30', '18h45', '06h00', '10h00', '10h44', '13h19',
       'Midday', '13h30', '10h45', '11h20', '11h45', '19h30', '08h30',
       '15h45', 'Shortly before 12h00', '17h34', '17h10', '11h15',
       '08h50', '17h45', '13h00', '10h20', '13h20', '02h00', '09h50',
       '11h30', '17h30', '9h00', '10h43', 'After noon', '15h15', '15h40',
       '19h05', '1300', '14h30 / 15h30', '22h00', '16h20', '14h34',
       '15h25', '14h55', '17h46', 'Morning ', '15h49', '19h00',
       'Midnight', '09h30 / 10h00', '10h15', '18h15', '04h00', '14h50',
       '13h50', '19h20', '10h25', '10h45-11h15', '16h45', '15h52',
       '06h15', '14h

In [6]:
df= df.dropna(how='all')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8703 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null  

In [8]:
df['Time'] = df['Time'].str.replace('07h00|07h30|08h00|08h30|09h00|09h30','Morning')
df['Time'] = df['Time'].str.replace('Early morning','Morning')
df['Time'] = df['Time'].str.replace('10h00|10h30|11h00|11h30|12h00|12h30','Late Morning')
df['Time'] = df['Time'].str.replace('Midday','Late Morning')
df['Time'] = df['Time'].str.replace('13h00|13h30|14h00|14h30|15h00|15h30','Afternoon')
df['Time'] = df['Time'].str.replace('Early afternoon','Afternoon')
df['Time'] = df['Time'].str.replace('16h00|16h30|17h00|17h30|18h00|18h30','Late afternoon')

# Padronizamos os horários de 7h às 9h30 como Morning, das 10h às 12h30 como Late Morning, 
# das 13h às 15h30 como Afternoon e das 16h às 18h30 como Late Afternoon. Além disso substituímos também as expressões que 
# mais apareciam e que poderíamos encaixar em uma das 4 classificações: 'Early morning','Morning', 'Midday','Late Morning', 
# 'Early afternoon','Afternoon'. 

<ipython-input-8-de09d68ded7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = df['Time'].str.replace('07h00|07h30|08h00|08h30|09h00|09h30','Morning')
<ipython-input-8-de09d68ded7f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = df['Time'].str.replace('Early morning','Morning')
<ipython-input-8-de09d68ded7f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [9]:
# Uma outra maneira de fazer isso: utilizamos apply e lambda para que em todos os itens da coluna Time nos seguintes horários 
# 07h, 7h, 08h, 8h, 09h e 9h sejam substituídos pela palavra Morning:
df['Time'] = df['Time'].apply(lambda x: re.sub('(0?[6789][Hh]\d\d)','Morning',str(x)))

# Utilizamos apply e lambda para que em todos os itens da coluna Time nos seguintes horários 10h, 11h e 12h 
#sejam substituidos pelas palavras Late Morning:
df['Time'] = df['Time'].apply(lambda x: re.sub('(1[012][Hh]\d\d)','Late Morning',str(x)))

# Utilizamos apply e lambda para que em todos os itens da coluna Time nos seguintes horários 13h, 14h e 14h 
# sejam substituidos pela palavra Afternoon:
df['Time'] = df['Time'].apply(lambda x: re.sub('(1[345][Hh]\d\d)','Afternoon',str(x)))

# Utilizamos apply e lambda para que em todos os itens da coluna Time nos seguintes horários 16h, 17h e 18h 
# sejam substituidos pelas palavras Late afternoon:
df['Time'] = df['Time'].apply(lambda x: re.sub('(1[678][Hh]\d\d)','Late afternoon',str(x)))

<ipython-input-9-cb90b82387e8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = df['Time'].apply(lambda x: re.sub('(0?[6789][Hh]\d\d)','Morning',str(x)))
<ipython-input-9-cb90b82387e8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = df['Time'].apply(lambda x: re.sub('(1[012][Hh]\d\d)','Late Morning',str(x)))
<ipython-input-9-cb90b82387e8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [10]:
# E uma terceira forma de fazer isso é utilizando uma função. Aqui criamos um dicionário com Regex e String e caso o Regex tenha
# match com a key, a função retorna uma das 4 segmentações que usamos também acima: Morning, Late Morning, Afternoon 
# e Late Afternoon.

def time_treat(texto):
    regex_dict = {'(0[6789][Hh]\d\d)':'Morning','(1[012][Hh]\d\d)':'Late Morning','(1[345][Hh]\d\d)': 'Afternoon','(1[678][Hh]\d\d)':'Late Afternoon','(1[9][Hh]\d\d)':'Night','(2[0123][Hh]\d\d)':'Night','Midday':'Late Morning','Early afternoon':'Afternoon','Early morning':'Morning'}
    
    for key,value in regex_dict.items():
        
        if re.match(key,str(texto).strip()):
            return value
        else:
            pass
    return texto


df['Time'] = df['Time'].apply(lambda x: time_treat(x))

<ipython-input-10-b47508969e30>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = df['Time'].apply(lambda x: time_treat(x))


In [11]:
# Contamos os resultados após a limpeza. 
df['Time'].value_counts(normalize=True)

nan                           0.661266
Afternoon                     0.098817
Late Morning                  0.073078
Morning                       0.052166
Late afternoon                0.052166
                                ...   
Late Morning / Afternoon      0.000115
Late afternoon or 1Morning    0.000115
"After lunch"                 0.000115
Morning / Afternoon           0.000115
Afternoon / Afternoon         0.000115
Name: Time, Length: 135, dtype: float64

In [12]:
# Reparamos na grande qtde de NAN, mas olhando os demais dados, não conseguimos inferir mais ajustes na coluna Time.
df[df['Time'] == 'nan']

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,...,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,...,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
11,2018.05.21,21-May-2018,2018.0,Unprovoked,USA,South Carolina,"Isle of Palms, Charleston County",Boogie boarding,Trey de Boer,M,...,NaN,"C. Creswell, GSAF",2018.05.21-deBoer.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.21,2018.05.21,6292.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_ranking = df['Time'].value_counts()
df_ranking.head(30)

# Aqui usamos o head com os 30 primeiros valores para uma última tentativa se há mais algum campo a ser considerado na limpeza. 
# Porém os demais são à noite com números que não alcaçam Late afternoon ou são muito pequenos ou não definidos o suficiente 
# para entrar em uma das 4 principais classificações de horário. 

nan                            5755
Afternoon                       860
Late Morning                    636
Morning                         454
Late afternoon                  454
1Morning                        191
Night                           110
Evening                          34
Dusk                             15
A.M.                             12
P.M.                             12
03h00                             6
05h00                             5
--                                5
02h00                             5
04h00                             5
Midnight                          4
Sunset                            4
01h00                             3
Morning - Morning                 3
Dark                              2
Just before noon                  2
Morning                           2
                                  2
Late Morning / Late Morning       2
05h30                             2
AM                                2
1600                        

#####  1) Quais horários os ataques acontecem mais? Há algum horário em que devemos aumentar a vigilância e/ou reforçar a equipe de guarda-costas das praias?

#####  R.: Os horários em que os acidentes são mais frequentes são no final da manhã e no início da tarde (entre 10h e 15h30). Esse é o horário em que sugerimos reforço de atenção e aumento da equipe.

In [14]:
df['Sex '].value_counts()

# Respondendo a segunda pergunta, é relevante sim a atenção adicional com o sexo masculino.
# pois as ocorrências são muito mais frequentes. 

M      5094
F       637
N         2
M         2
lli       1
.         1
Name: Sex , dtype: int64

In [15]:
df['Activity'].value_counts(normalize=True)

# Seriam os surfistas os mais atacados? Aparentemente sim! Vamos investigar melhor. Abaixo mostra que a maioria das pessoas (quase 17%) realmente estava surfando 
# quando foram atacadas. 

Surfing                                                      0.168635
Swimming                                                     0.150920
Fishing                                                      0.074852
Spearfishing                                                 0.057833
Bathing                                                      0.028135
                                                               ...   
3-masted steel barque Glenbank foundered during a cyclone    0.000174
Ship lay at anchor & man was working on its rudder           0.000174
Fishing from 32' boat                                        0.000174
Wading / fishing & carrying a bag of fish                    0.000174
Standing on ship deck                                        0.000174
Name: Activity, Length: 1532, dtype: float64

In [16]:
agg = df.groupby(['Sex ', 'Activity'])[['Case Number']].count().sort_values(by='Case Number', ascending=False)
agg
# Agrupando gênero e atividade, vemos que no total geral a maioria são surfistas e quando separamos por sexo, vemos 
# que a maioria mesmo está concentrada no sexo masculino.

Case Number
Sex  Activity                                           
M    Surfing                                         890
     Swimming                                        691
     Fishing                                         329
     Spearfishing                                    312
F    Swimming                                        154
...                                                  ...
M    Fishing  (trawling)                               1
     Fisherman                                         1
     Finning the shark                                 1
     Filming underwater, carrying powerhead            1
lli  Surfing                                           1

[1448 rows x 1 columns]

#### 2) É relevante uma atenção adicional por gênero (M / F) de banhistas?
#### R.: Sim. A grande maioria de ataques ocorrem com homens surfando (principalmente) e nadando.

#### 3) E por país, é necessária uma atenção maior também?

In [17]:
df['Country'].value_counts()

USA                           2229
AUSTRALIA                     1338
SOUTH AFRICA                   579
PAPUA NEW GUINEA               134
NEW ZEALAND                    128
                              ... 
GULF OF ADEN                     1
GABON                            1
INDIAN OCEAN?                    1
ANDAMAN / NICOBAR ISLANDAS       1
AFRICA                           1
Name: Country, Length: 212, dtype: int64

#### R.: Sim, a grande quantidade de ocorrências são maiores na Austrália e Estados Unidos (melhores países para surfar!). 

In [18]:
df_coun = df.groupby(by=['Country', 'Sex '])[['Case Number']].count()
df_coun.sort_values(by=['Case Number'], inplace= True, ascending=False)
df_coun.head(20)

,,Case Number
Country,Sex,
USA,M,1791
AUSTRALIA,M,1107
SOUTH AFRICA,M,489
USA,F,337
PAPUA NEW GUINEA,M,108
NEW ZEALAND,M,100
BRAZIL,M,93
BAHAMAS,M,92
AUSTRALIA,F,88


In [19]:
df_coun = df.groupby(by=['Time', 'Sex '])[['Case Number']].count()
df_coun.sort_values(by=['Case Number'], inplace= True, ascending=False)
df_coun.head(20)

,,Case Number
Time,Sex,
nan,M,2637
Afternoon,M,703
Late Morning,M,531
Morning,M,389
Late afternoon,M,366
nan,F,252
1Morning,M,165
Afternoon,F,130
Night,M,97


## Conclusão:

#### Sugerimos então o aumento de equipe de guarda costas nos horários entre 10h e 15h30. Como a grande maioria dos acidentes ocorre com homens surfirstas, recomendamos também a contratação de guarda-costas homens, para facilitar qualquer resgate necessário. Esse reporte será compartilhado com todos os países com praias próprias para a prática do surf, mas reforçaremos essas necessidades com os países EUA e Austrália, que são os de maior incidência de acidentes.  

In [20]:
#Exportando novo arquivo:
df.to_csv('attacks_new.csv', encoding='latin-1', index= False)